# Sudoku
*Tomáš Kalvoda, KAM FIT ČVUT, 2015, Notebook vytvořen pomocí Sage 6.6.rc2*

## Sudoku v *Sage*

Všichni jistě známe hlavolam Sudoku. *Sage* má funkce pro jejich zobrazování a řešení.

In [2]:
# konstruktoru lze předat i matici
sudoku = Sudoku('451.3...........7....9.14.8.1....6.3.9..5..1.7.3....8.5.97.4....2...........8.195')
print sudoku

+-----+-----+-----+
|4 5 1|  3  |     |
|     |     |  7  |
|     |9   1|4   8|
+-----+-----+-----+
|  1  |     |6   3|
|  9  |  5  |  1  |
|7   3|     |  8  |
+-----+-----+-----+
|5   9|7   4|     |
|  2  |     |     |
|     |  8  |1 9 5|
+-----+-----+-----+


Sudoku nemusí mít jednoznačné řešení (pokud vůbec existuje). Nalezněme jedno z nich.

In [3]:
timeit('next(sudoku.solve())')

625 loops, best of 3: 1.36 ms per loop


In [4]:
print next(sudoku.solve())

+-----+-----+-----+
|4 5 1|8 3 7|9 6 2|
|9 6 8|2 4 5|3 7 1|
|2 3 7|9 6 1|4 5 8|
+-----+-----+-----+
|8 1 5|4 7 9|6 2 3|
|6 9 2|3 5 8|7 1 4|
|7 4 3|1 2 6|5 8 9|
+-----+-----+-----+
|5 8 9|7 1 4|2 3 6|
|1 2 6|5 9 3|8 4 7|
|3 7 4|6 8 2|1 9 5|
+-----+-----+-----+


## Řešení Sudoku pomocí lineárního programování

Sudoku můžeme řešit i pomocí lienárního programování. Řešení Sudoku lze zformulovat jako úlohu lineárního programování.

In [5]:
# zadání
mat = sudoku.to_matrix()
print mat

[4 5 1 0 3 0 0 0 0]
[0 0 0 0 0 0 0 7 0]
[0 0 0 9 0 1 4 0 8]
[0 1 0 0 0 0 6 0 3]
[0 9 0 0 5 0 0 1 0]
[7 0 3 0 0 0 0 8 0]
[5 0 9 7 0 4 0 0 0]
[0 2 0 0 0 0 0 0 0]
[0 0 0 0 8 0 1 9 5]


Sudoku je mřížka $9\times 9$ čísel od $1$ do $9$. Podmínkou je, aby se v každém řádku, každém sloupci každém $3\times 3$ bloku (viz výše) vyskytovalo každé z čísel $1$ až $9$ právě jednou. Abychom všechny tyto podmínky mohli formulovat, použijeme k popisu problému trojrozměrné pole (tenzor třetího řádu) $s_{i,j,k}$, $i,j,k = 1,2,\ldots,9$. První dva indexy $i$ a $j$ určují polohu Sudoku tabulce a třetí index $k$ označuje cifru, která v příslušném políčku leží, v takovém případě klademe $s_{i,j,k} = 1$, jinak $0$.

Nyní sestavme podmínky na správné řešení. Nejprve poznamenejme, že $s_{i,j,k} \in \{0,1\}$ pro libovolné $i,j,k=1,2,\ldots,9$.
 - Nejprve zafixujeme zadání, klademe
   $$ s_{i,j,k} = 1 $$
   kdykoliv na pozici $i,j$ zadání je cifra $k$.
 - V každém políčku je právě jedna cifra,
   $$ \sum_{k=1}^9 s_{i,j,k} = 1 $$
   pro každé $i,j = 1,2,\ldots, 9$.
 - V každém řádku se všechny cifry vyskytují právě jednou, tedy
   $$ \sum_{j=1}^9 s_{i,j,k} = 1 $$
   pro každé $i,k=1,2,\ldots,9$.
 - Stejným způsobem jako výše požadujeme, aby se v každém sloupci vyskytovala každá z cifer právě jednou.
 - V každém z $3 \times 3$ bloku se každá cifra musí vyskytovat právě jednou:
   $$ \sum_{i=u}^{u+2} \sum_{j=v}^{v+2} s_{i,j,k} = 1 $$
   pro každé $k=1,2,\ldots,9$ a $u,v = 1,4,7$.

Objektivní funkci není potřeba specifikovat, pouze se snažíme zjistit, jestli výše uvedená sada podmínek má řešení.

In [12]:
n = 9
p = MixedIntegerLinearProgram(maximization=True)
y = p.new_variable(binary=True)
# objektivní funkce není potřeba
#p.set_objective(sum([k*y[(i,j,k)] for i in range(n) for j in range(n) for k in range(1,n+1)]))

# zadání
for i,j in mrange([n,n]):
    if mat[i,j] > 0: p.add_constraint(y[(i,j,mat[i,j])] == 1)

# v každém políčku je právě jedna cifra
for i,j in mrange([n,n]):
    p.add_constraint(sum([y[(i,j,k)] for k in range(1,n+1)]) == 1)

# v každém řádku se všechny cifry vyskytují právě jednou
for i in range(n):
    for k in range(1,n+1):
        p.add_constraint(sum([y[(i,j,k)] for j in range(n)]) == 1)

# v každém sloupci se všechny cifry vyskytují právě jednou
for j in range(n):
    for k in range(1,n+1):
        p.add_constraint(sum([y[(i,j,k)] for i in range(n)]) == 1)

# subarrays unique
for k in [0,3,6]:
    for l in [0,3,6]:
        for u in range(1,n+1):
            p.add_constraint(
                sum([y[(i,j,u)] for i in range(k,k+3) for j in range(l,l+3)]) == 1
            )

Celkem máme následující počet podmínek:

In [13]:
p.number_of_constraints()

351

A následující počet proměnných.

In [14]:
p.number_of_variables()

729

Nyní problém vyřešme!

In [15]:
timeit('p.solve()')

625 loops, best of 3: 793 µs per loop


Extrahujme matici s výsledky.

In [16]:
out = matrix(ZZ,n,n)
for i,j in mrange([n,n]):
    for k in range(1,10):
        if p.get_values(y[(i,j,k)]) == 1: out[i,j] = k

A nechme si ji pěkně zobrazit.

In [17]:
Sudoku(out)

+-----+-----+-----+
|4 5 1|8 3 7|9 6 2|
|9 6 8|2 4 5|3 7 1|
|2 3 7|9 6 1|4 5 8|
+-----+-----+-----+
|8 1 5|4 7 9|6 2 3|
|6 9 2|3 5 8|7 1 4|
|7 4 3|1 2 6|5 8 9|
+-----+-----+-----+
|5 8 9|7 1 4|2 3 6|
|1 2 6|5 9 3|8 4 7|
|3 7 4|6 8 2|1 9 5|
+-----+-----+-----+